In [1]:
from scripts.dataMerging.combineDatasets import extract_features_elevation , extract_features_landcover , extract_features_monthly_clim , extract_features_soil , organize_monthly_climat_files
from scripts.tools.merge_data_sources import merge_data_sources





# Merging Data

## Climat Dataset



In [2]:

# Organize the files
monthly_tmax_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/max/*.tif"
)
monthly_tmin_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/min/*.tif"
)
monthly_tprec_data = organize_monthly_climat_files(
    "../data/climate_dataset/5min/prec/*.tif"
)


fires_tmax = extract_features_monthly_clim(
    fire_csv="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_dict=monthly_tmax_data,
    output_path="../data/features/fire_tmax.csv",
    value_name="tmax",
)


fires_tmin = extract_features_monthly_clim(
    fire_csv="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_dict=monthly_tmin_data,
    output_path="../data/features/fire_tmin.csv",
    value_name="tmin",
)


fires_tprec = extract_features_monthly_clim(
    fire_csv="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_dict=monthly_tprec_data,
    output_path="../data/features/fire_tprec.csv",
    value_name="tprec",
)

✅ Extracted tmax for month 01 (6355 points)
✅ Extracted tmax for month 02 (5469 points)
✅ Extracted tmax for month 03 (8783 points)
✅ Extracted tmax for month 04 (6687 points)
✅ Extracted tmax for month 05 (7623 points)
✅ Extracted tmax for month 06 (7030 points)
✅ Extracted tmax for month 07 (8664 points)
✅ Extracted tmax for month 08 (8973 points)
✅ Extracted tmax for month 09 (7051 points)
✅ Extracted tmax for month 10 (8619 points)
✅ Extracted tmax for month 11 (8211 points)
✅ Extracted tmax for month 12 (6785 points)
💾 Saved to ../data/features/fire_tmax.csv
✅ Extracted tmin for month 01 (6355 points)
✅ Extracted tmin for month 02 (5469 points)
✅ Extracted tmin for month 03 (8783 points)
✅ Extracted tmin for month 04 (6687 points)
✅ Extracted tmin for month 05 (7623 points)
✅ Extracted tmin for month 06 (7030 points)
✅ Extracted tmin for month 07 (8664 points)
✅ Extracted tmin for month 08 (8973 points)
✅ Extracted tmin for month 09 (7051 points)
✅ Extracted tmin for month 10 (861

## Landcover

In [3]:

fires_with_landcover = extract_features_landcover(
    csv_path="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    shapefile_path="../data/land_dataset/combined/alg_tun_landcvr.shx",
    lat_col="latitude",
    lon_col="longitude",
    keep_cols=["GRIDCODE"],  # can be ["GRIDCODE", "CLASS", "AREA", ...]
    output_path="../data/features/fire_landcover.csv",
)


## SOil

In [6]:
fires_with_soil = extract_features_soil(
    csv_path="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    raster_path="../data/soil_dataset/original/HWSD2_RASTER/HWSD2.bil",
    soil_attributes_csv="../data/soil_dataset/simplified/D1_soil_features_alg_tun.csv",
    output_soil_ids="../data/features/fire_soil_ids.csv",
    output_soil_feature="../data/features/fire_soil.csv",
)

## elevation

In [7]:
fires_with_elevation = extract_features_elevation(
    raster_path="../data/elevation_dataset/elevation_clipped.tif",
    fire_csv_path="../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv",
    output_csv="../data/features/fire_elevation.csv",
    value_name="elevation",
)


Loaded 90250 points from ../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv


Extracting elevation: 100%|██████████| 90250/90250 [00:06<00:00, 14606.00it/s]


✅ Saved extracted values to ../data/features/fire_elevation.csv


## Merging

In [17]:
sources = ["../data/fire_dataset/viirs-jpss1_2024_alg_Tun.csv","../data/features/fire_tmin.csv" , "../data/features/fire_tmax.csv" , "../data/features/fire_tprec.csv"]
temp_df = merge_data_sources(sources , ["latitude","longitude","acq_date"], "inner")

csv_list= [temp_df,  "../data/features/fire_landcover.csv" , "../data/features/fire_elevation.csv" , "../data/features/fire_soil.csv"]
temp_df = merge_data_sources(
    csv_list,
    on=["latitude", "longitude"],
    how="inner",
    output_path="../data/Merged/merged.csv"
)


Merged CSV saved to: ../data/Merged/merged.csv
